# <center> MATH&ML-15. Рекомендательные системы. Часть II

In [26]:
#Импорты

import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты
from surprise.model_selection import train_test_split
from surprise import SVD, KNNBasic, accuracy

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

### Юнит 2. Content-based model

In [27]:
netflix_titles = pd.read_csv('data/netflix_titles.zip')
netflix_titles

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


In [28]:
netflix_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [29]:
#Обучаем модель TF-IDF
model = TfidfVectorizer(stop_words='english')

netflix_titles['description'] = netflix_titles['description'].fillna('')
feature_matrix = model.fit_transform(netflix_titles['description'])

feature_matrix.shape

(7787, 17905)

In [30]:
#Вычсляем косинусную близость
cosine_sim = linear_kernel(feature_matrix, feature_matrix)
#В итоге у нас матрица. Вернем для удобства ей индексы
indices = pd.Series(netflix_titles.index,index=netflix_titles['title']).drop_duplicates()

In [31]:
#Функция для создания рекомендаций
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return netflix_titles['title'].iloc[ind_movie]

get_recommendations('Star Trek')

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

In [32]:
#Задание 2.3
get_recommendations('Balto')

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

### Юнит 3. Коллаборативная фильтрация

In [33]:
data = Dataset.load_from_file(
    "data/u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])
display(df)

,userId,movieId,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


userId — идентификаторы пользователей сайта movielens;  
movieId — идентификаторы фильмов;  
rating — оценки фильмов, выставленные пользователями по шкале от 1 до 5;  
timestamp — время оценки фильма пользователем. Данный формат представления времени показывает, сколько секунд прошло с 1 января 1970 года.

In [34]:
#Задание 3.1
print('Всего фильмов в выборке:', len(df['movieId'].value_counts()))

Всего фильмов в выборке: 1682


In [35]:
#Задание 3.2
print('Всего пользователей в выборке:', len(df['userId'].value_counts()))

Всего пользователей в выборке: 943


In [36]:
#Задание 3.3
print('Топ оценок:')
display(df['rating'].value_counts())

Топ оценок:


rating
4.0    34174
3.0    27145
5.0    21201
2.0    11370
1.0     6110
Name: count, dtype: int64

In [37]:
#Задание 3.4

#Судя по документации, принимаются только изначальные данные, не датафрейм
train_df, test_df = train_test_split(data, test_size=0.25, random_state=13)
display(test_df[:3])
print('Количество объектов в тестовой выборке:', len(test_df))

[('7', '633', 5.0), ('422', '287', 3.0), ('804', '163', 3.0)]

Количество объектов в тестовой выборке: 25000


In [38]:
#Базовая коллаборативная фильтрация item-based

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)
knn.fit(train_df)

predictions = knn.test(test_df)
display(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.


[Prediction(uid='7', iid='633', r_ui=5.0, est=4.199452349030111, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='422', iid='287', r_ui=3.0, est=3.4703437660463736, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='804', iid='163', r_ui=3.0, est=3.5716736533692854, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='189', iid='480', r_ui=5.0, est=4.222825780855538, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='238', iid='546', r_ui=3.0, est=3.473417286928204, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid='804', iid='216', r_ui=4.0, est=3.922551907749182, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='350', iid='204', r_ui=4.0, est=4.345238219480267, details={'actual_k': 38, 'was_impossible': False}),
 Prediction(uid='708', iid='993', r_ui=4.0, est=3.4458505791534115, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='193', iid='1078', r_ui=4.0, es

In [39]:
#Для предсказания какого-то определенного объекта необходимо помнить, 
# что все данные, кроме реального рейтинга представленные в виде object

#Задание 3.5
display(df[(df['userId']=='500') & (df['movieId']=='699')])
pred = knn.predict(uid='500', iid='699', r_ui=3)
pred

,userId,movieId,rating,timestamp
34367,500,699,3.0,883875523


Prediction(uid='500', iid='699', r_ui=3, est=3.4747900407148165, details={'actual_k': 40, 'was_impossible': False})

In [40]:
print(accuracy.rmse(predictions))

pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

#Пример вывода рекомендаций
recom = pred[pred.uid =='849']['iid'].to_list()
print('movieId для данного пользователя:', recom)

RMSE: 1.0272
1.0271678039029761
movieId для данного пользователя: ['234', '427', '568', '174']


In [41]:
#Задание 3.6
#Базовая коллаборативная фильтрация user-based

sim_options = {
    'name': 'cosine',
    'user_based': True
}
 
knn = KNNBasic(sim_options=sim_options)
knn.fit(train_df)

predictions = knn.test(test_df)
display(predictions)

print(accuracy.rmse(predictions))

pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

#Пример вывода рекомендаций
recom = pred[pred.uid =='849']['iid'].to_list()
print('movieId для данного пользователя:', recom)

Computing the cosine similarity matrix...
Done computing similarity matrix.


[Prediction(uid='7', iid='633', r_ui=5.0, est=4.150997362033697, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='422', iid='287', r_ui=3.0, est=3.754016750770759, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='804', iid='163', r_ui=3.0, est=3.70096140390557, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='189', iid='480', r_ui=5.0, est=4.524549229206855, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='238', iid='546', r_ui=3.0, est=3.1991812561559425, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='804', iid='216', r_ui=4.0, est=3.9500160918588327, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='350', iid='204', r_ui=4.0, est=4.1983705425936275, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='708', iid='993', r_ui=4.0, est=3.6795202365264075, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='193', iid='1078', r_ui=4.0, es

RMSE: 1.0175
1.0174852296380237
movieId для данного пользователя: ['427', '174', '568', '234']


In [42]:
knn = SVD()
knn.fit(train_df)

predictions = knn.test(test_df)
display(predictions)

print(accuracy.rmse(predictions))

pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

#Пример вывода рекомендаций
recom = pred[pred.uid =='849']['iid'].to_list()
print('movieId для данного пользователя:', recom)

[Prediction(uid='7', iid='633', r_ui=5.0, est=4.64024779258598, details={'was_impossible': False}),
 Prediction(uid='422', iid='287', r_ui=3.0, est=3.837707787547249, details={'was_impossible': False}),
 Prediction(uid='804', iid='163', r_ui=3.0, est=3.716589498709493, details={'was_impossible': False}),
 Prediction(uid='189', iid='480', r_ui=5.0, est=4.590363701128451, details={'was_impossible': False}),
 Prediction(uid='238', iid='546', r_ui=3.0, est=2.8808099768401343, details={'was_impossible': False}),
 Prediction(uid='804', iid='216', r_ui=4.0, est=4.315455648482262, details={'was_impossible': False}),
 Prediction(uid='350', iid='204', r_ui=4.0, est=3.7727455398412943, details={'was_impossible': False}),
 Prediction(uid='708', iid='993', r_ui=4.0, est=3.6888715264257623, details={'was_impossible': False}),
 Prediction(uid='193', iid='1078', r_ui=4.0, est=3.0454415500590204, details={'was_impossible': False}),
 Prediction(uid='847', iid='173', r_ui=5.0, est=3.957082021835286, deta

RMSE: 0.9434
0.9433567610472382
movieId для данного пользователя: ['174', '427', '568', '234']


### Юнит 5. Современные методы: глубокое обучение

In [43]:
print(tf.__version__)

2.20.0


In [44]:
df = pd.read_csv('data/Gooddreadbooks/ratings.csv')
display(df.head())
display(df.info())

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981756 entries, 0 to 981755
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   book_id  981756 non-null  int64
 1   user_id  981756 non-null  int64
 2   rating   981756 non-null  int64
dtypes: int64(3)
memory usage: 22.5 MB


None

In [47]:
#Задание 5.1

train, test = sklearn.model_selection.train_test_split(df, test_size=0.2, random_state=42)
print('Количество объектов в обучающей выборке:', train.shape[0])

Количество объектов в обучающей выборке: 785404


In [48]:
#Задание 5.2

n_books = df['book_id'].nunique()
print('Количество уникальных книг в данных:', n_books)

Количество уникальных книг в данных: 10000


In [49]:
#Задание 5.3

n_users = df['user_id'].nunique()
print('Количество уникальных пользователей в данных:', n_users)

Количество уникальных пользователей в данных: 53424


In [50]:
#Эмбеддинги и векторы для книг и пользователей

book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

#Соединим векторы
conc = Concatenate()([book_vec, user_vec])

#Собираем нейросеть
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

model2 = Model([user_input, book_input], out)

#Добавляем оптимизатор и метрику обучения
model2.compile(optimizer = 'adam', loss = 'mean_squared_error')

#Обучение модели
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

#Оценка качества
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 0.8012
Epoch 2/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 0.6872
Epoch 3/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 0.6570
Epoch 4/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 0.6348
Epoch 5/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.6153
6136/6136 ━━━━━━━━━━━━━━━━━━━━ 5s 729us/step - loss: 0.7070


0.707029402256012

In [51]:
#Добавим ещё слой, больше эпох и обучим заново
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 0.6262
Epoch 2/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 0.5820
Epoch 3/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 46s 2ms/step - loss: 0.5611
Epoch 4/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - loss: 0.5435
Epoch 5/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - loss: 0.5289
Epoch 6/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 36s 1ms/step - loss: 0.5170
Epoch 7/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 44s 2ms/step - loss: 0.5065
Epoch 8/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - loss: 0.4974
Epoch 9/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 37s 1ms/step - loss: 0.4895
Epoch 10/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 37s 1ms/step - loss: 0.4824
6136/6136 ━━━━━━━━━━━━━━━━━━━━ 5s 740us/step - loss: 0.7763


0.7763336896896362